# Spaceship Titanic

In [1]:
import pandas as pd
import numpy as npy
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
train = pd.read_csv('../data/train.csv')

In [15]:
test = pd.read_csv('../data/test.csv')

In [18]:
test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [6]:
data_passenger = \
pd.concat([train.PassengerId.str.split('_', expand = True), 
           train.Transported], axis = 1)

data_passenger = data_passenger[[0, 1, 'Transported']].copy()
data_passenger.columns = ['Id', 'Id2', 'Transported']

In [7]:
data_train =  \
pd.get_dummies(train, 
               columns = ['HomePlanet', 'Destination'], 
               prefix = ['HP','D'], drop_first = True)\
    .drop(columns = ['Name'])

data_train = \
pd.concat([data_train, train.Cabin.str.split('/', expand = True), ], axis = 1,)

data_train.rename(columns = {0:'deck', 1:'num', 2: 'side'}, inplace = True)
data_train = \
    pd.get_dummies(data_train,
                   columns = ['deck', 'side'], prefix = ['Deck', 'Side'],
                   drop_first = False)
data_train.drop(columns = ['num'], inplace = True)

In [8]:
data_train.RoomService = [0 if x == 0 else 1 for x in data_train.RoomService]
data_train.FoodCourt = [0 if x == 0 else 1 for x in data_train.FoodCourt]
data_train.ShoppingMall = [0 if x == 0 else 1 for x in data_train.ShoppingMall]
data_train.Spa = [0 if x == 0 else 1 for x in data_train.Spa]
data_train.VRDeck = [0 if x == 0 else 1 for x in data_train.VRDeck]

In [9]:
data_train =  \
data_train\
    .set_index('PassengerId')\
    .replace({True:1, False: 0})\
    .drop(columns= ['Cabin'])\
    .dropna()\
    .copy()

In [10]:
X_train = data_train.drop(columns = ['Transported', 'Age'])
y_train = data_train['Transported']

In [13]:
X_train.CryoSleep = X_train.CryoSleep.astype(int)
X_train.VIP = X_train.VIP.astype(int)

,CryoSleep,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HP_Europa,HP_Mars,D_PSO J318.5-22,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
0002_01,0,0,1,1,1,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
0003_01,0,1,1,1,0,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,1
0003_02,0,0,0,1,1,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,1
0004_01,0,0,1,1,1,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [21]:
pd.concat([X_train, y_train], axis = 1).to_csv('../data/clean_train.csv')

In [24]:
salida_ = []
for i in range(X_train.shape[0]):
    valor_ = 0
    for j in range(X_train.shape[1]):
        valor_ = valor_ + X_train.iloc[i,j] * 2 **j
    salida_.append(valor_)

In [34]:
pd.DataFrame(salida_, index = X_train.index)\
    .merge(y_train, left_index = True, right_index = True)

,0,Transported
PassengerId,,
0001_01,529536,0
0002_01,1115260,1
0003_01,1051886,0
0003_02,1051896,0
0004_01,1115260,1
...,...,...
9276_01,526570,0
9278_01,1180161,0
9279_01,1180720,1
